# Prioritização de Candidatos com Machine Learning

Este projeto foi desenvolvido para auxiliar o time de recrutamento a identificar candidatos com maior probabilidade de contratação, utilizando dados históricos de processos seletivos.

- **Objetivo**: Priorizar automaticamente os candidatos com maior potencial de contratação.
- **Abordagem**: Modelos supervisionados com foco em recall da classe contratada (target = 1).
- **Modelo final**: XGBoost com balanceamento de classes e variável `cliente` incluída.


## Comparação entre modelos

| Modelo                          | Accuracy | Precision (Classe 1) | Recall (Classe 1) | F1-score (Classe 1) |
|--------------------------------|----------|-----------------------|-------------------|---------------------|
| XGBoost com `cliente`          | 0.75     | 0.50                  | 0.54              | 0.52                |
| RandomForest balanceado        | 0.76     | 0.50                  | 0.54              | 0.52                |
| XGBoost sem `cliente`          | 0.56     | 0.29                  | 0.56              | 0.38                |
| RandomForest original          | 0.78     | 0.58                  | 0.39              | 0.47                |

➡️ O modelo final escolhido foi o **XGBoost com `cliente`**, por apresentar o melhor equilíbrio entre recall da classe positiva e robustez na base atual.

In [7]:
import pandas as pd
import joblib

X_test = pd.read_csv("../data/processed/X_test.csv")

# pipeline_xgb deve estar treinado previamente
pipeline_xgb = joblib.load("../models/modelo_priorizacao_xgb.pkl")

In [8]:
def priorizar_candidatos(df_input, modelo, top_n=10):
    """
    Recebe um DataFrame de candidatos e retorna os top_n com maior probabilidade de contratação.
    """
    df_copy = df_input.copy()
    df_copy = df_copy.drop(columns=["target"], errors="ignore")

    # Previsão
    probs = modelo.predict_proba(df_copy)[:, 1]
    df_copy["prob_contratacao"] = probs

    # Ordenação
    df_ranked = df_copy.sort_values(by="prob_contratacao", ascending=False)
    return df_ranked.head(top_n)

In [11]:
# Simulação de uso
top_priorizados = priorizar_candidatos(X_test.copy(), pipeline_xgb, top_n=5)
top_priorizados

,nivel_academico,ingles,espanhol,area_atuacao,nivel_profissional,sap,cliente,prob_contratacao
732,Ensino Superior Cursando,Intermediário,Básico,Ti - Sistemas E Ferramentas,Sênior,Não,Gonzalez And Sons,0.997305
554,Ensino Médio Completo,Avançado,Básico,Ti - Sap,Sênior,Não,Jenkins-Walker,0.987796
1189,Ensino Superior Completo,Básico,Básico,Ti - Sap,Pleno,Não,Miller-Curry,0.980813
1254,Ensino Superior Completo,Básico,Intermediário,Ti - Sap,Pleno,Não,Miller-Curry,0.979685
1958,Ensino Superior Completo,Básico,Nenhum,Ti - Projetos,Sênior,Não,Gonzalez And Sons,0.969872
